In [2]:
!git clone https://github.com/chuangchuangtan/NPR-DeepfakeDetection.git

Cloning into 'NPR-DeepfakeDetection'...


In [3]:
!cd "C:\\Users\\darth\\Desktop\\CV code Demo\\NPR-DeepfakeDetection"


In [6]:
!pip install -r "C:\\Users\\darth\\Desktop\\CV code Demo\\NPR-DeepfakeDetection\\requirements.txt"

^C


In [6]:
import streamlit as st
import torch
import torch.nn as nn
from torch.nn import functional as F
from torchvision.models import resnet50
from torchvision import transforms
from PIL import Image
import requests
from io import BytesIO
import numpy as np
import random
import os

In [7]:
def seed_torch(seed=1029):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.enabled = False

seed_torch(100)

In [8]:
NPRmodel = resnet50()
NPRmodel.fc1 = nn.Linear(512, 1)
NPRmodel.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1, bias=False)
del NPRmodel.layer3, NPRmodel.layer4, NPRmodel.fc

In [13]:
def load_model():
    NPR_url = "C:\\Users\\darth\\Desktop\\CV code Demo\\NPR-DeepfakeDetection\\model_epoch_last_3090.pth"#'https://raw.githubusercontent.com/chuangchuangtan/NPR-DeepfakeDetection/main/model_epoch_last_3090.pth'
    state_dict = torch.hub.load_state_dict_from_url(NPR_url, map_location='cpu')
    NPRmodel.load_state_dict(state_dict, strict=True)
    NPRmodel.eval()
    return NPRmodel

In [14]:
trans = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [15]:
def interpolate(img, factor):
    return F.interpolate(F.interpolate(img, scale_factor=factor, mode='nearest', recompute_scale_factor=True),
                         scale_factor=1/factor, mode='nearest', recompute_scale_factor=True)

In [16]:
def predict_image(img, model):
    img = trans(img.convert('RGB')).unsqueeze(0)
    _, c, w, h = img.shape
    if w % 2 == 1: img = img[:, :, :-1, :]
    if h % 2 == 1: img = img[:, :, :, :-1]
    
    NPR = img - interpolate(img, 0.5)
    with torch.no_grad():
        x = model.conv1(NPR * 2.0 / 3.0)
        x = model.bn1(x)
        x = model.relu(x)
        x = model.maxpool(x)
        x = model.layer1(x)
        x = model.layer2(x).mean(dim=(2, 3), keepdim=False)
        x = model.fc1(x)
        pred = x.sigmoid().cpu().numpy()
    
    return {'Fake Image': float(pred), 'Real Image': 1.0 - float(pred)}

In [25]:
image = Image.open("C:\\Users\\darth\\Desktop\\CV code Demo\\CelebAHQ_00000110.png")

In [ ]:
C:\Users\darth\Desktop\CV code Demo\flux1.webp

In [26]:
model = load_model()

In [27]:
result = predict_image(image, model)

C:\Users\darth\AppData\Local\Temp\ipykernel_22252\2603384312.py:18: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return {'Fake Image': float(pred), 'Real Image': 1.0 - float(pred)}


In [28]:
result

{'Fake Image': 0.0, 'Real Image': 1.0}

In [3]:
NPR_url = 'https://raw.githubusercontent.com/chuangchuangtan/NPR-DeepfakeDetection/main/model_epoch_last_3090.pth'
state_dict = torch.hub.load_state_dict_from_url(NPR_url, map_location='cpu')

Downloading: "https://raw.githubusercontent.com/chuangchuangtan/NPR-DeepfakeDetection/main/model_epoch_last_3090.pth" to C:\Users\darth/.cache\torch\hub\checkpoints\model_epoch_last_3090.pth
100.0%


In [ ]:
!pip install -r /content/NPR-DeepfakeDetection/requirements.txt

In [4]:
import sys
sys.path.append('C:\\Users\\darth\\Desktop\\CV code Demo\\NPR-DeepfakeDetection')

In [5]:
import torch
from util import Logger, printSet
from validate import validate
from networks.resnet import resnet50
from options.test_options import TestOptions
import networks.resnet as resnet
import numpy as np
import random
import cv2
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt

# Load the model
opt = TestOptions().parse(print_options=False)
model = resnet50(num_classes=1)
state_dict = torch.load(opt.model_path, map_location='cpu')
model.load_state_dict(state_dict['model'])
#model.cuda()
model.eval()

# Define the preprocessing transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to 224x224
    transforms.ToTensor(),  # Convert the image to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
])

# Define a function for inference
def inference(image_path):
    # Load the image
    image = Image.open(image_path)

    # Preprocess the image
    image = transform(image)

    # Add batch dimension
    image = image.unsqueeze(0)

    # Move the image to the GPU
    image = image#.cuda()

    # Run the image through the model
    output = model(image)

    # Get the prediction
    prediction = torch.sigmoid(output)

    # Return the prediction
    return prediction

# Test the inference function
image_path = 'path/to/test/image.jpg'
prediction = inference(image_path)
print('Prediction:', prediction)

# Display the image
image = Image.open(image_path)
plt.imshow(image)
plt.show()

# Display the prediction
print('Prediction:', prediction.item())

AttributeError: module 'torch._C' has no attribute '_cuda_setDevice'

AttributeError: module 'torch._C' has no attribute '_cuda_setDevice'